In [ ]:
##################### CRIME DATA #################

In [57]:
import h5py
import numpy as np
hf = h5py.File('../data/features.h5', 'r')
# hf.keys()
X=np.array(hf["features"][:])
hf.close()
hf = h5py.File('../data/targets.h5', 'r')
# hf.keys()
y=np.array(hf["targets"][:])

In [58]:
print(X.shape)
print(y.shape)

(6965, 16, 6, 26, 26)
(6965, 6, 26, 26)


In [59]:
X=X.reshape(-1,26,26,16,6)
y=y.reshape(-1,26,26,6)
# y=np.expand_dims(y, axis=-2)

In [60]:
print(X.shape)
print(y.shape)

(6965, 26, 26, 16, 6)
(6965, 26, 26, 6)


In [62]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)

In [63]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(4457, 26, 26, 16, 6)
(4457, 26, 26, 6)
(1115, 26, 26, 16, 6)
(1115, 26, 26, 6)
(1393, 26, 26, 16, 6)
(1393, 26, 26, 6)


In [65]:
from tensorflow import keras
from tensorflow.keras import layers

In [66]:
time=16
x=26
y=26
channels=6
from keras.models import Sequential
inp=layers.Input(shape=(x,y,time,channels))
x=layers.ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                   , padding='same', return_sequences=True)(inp)
# x=layers.BatchNormalization()(x)
x=layers.MaxPooling3D(pool_size=(1, 1, 2), padding='same', data_format='channels_last')(x)
x=layers.ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                   , padding='same', return_sequences=True)(x)
# x=layers.BatchNormalization()(x)
x=layers.MaxPooling3D(pool_size=(1, 1, 2), padding='same', data_format='channels_last')(x)
x=layers.ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                   , padding='same', return_sequences=True)(x)
# x=layers.BatchNormalization()(x)
x=layers.MaxPooling3D(pool_size=(1, 1, 2), padding='same', data_format='channels_last')(x)
x=layers.ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                   , padding='same', return_sequences=True)(x)
# x=layers.BatchNormalization()(x)
x=layers.MaxPooling3D(pool_size=(1, 1, 2), padding='same', data_format='channels_last')(x)
x=layers.Conv3D(1, 
                     kernel_size = (3, 3, 1), 
                     padding = 'same',
                     activation = 'relu')(x)

model = keras.models.Model(inp, x)

In [67]:
print(model.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 26, 26, 16, 6)]   0         
                                                                 
 conv_lstm2d_16 (ConvLSTM2D)  (None, 26, 26, 16, 20)   18800     
                                                                 
 max_pooling3d_16 (MaxPoolin  (None, 26, 26, 8, 20)    0         
 g3D)                                                            
                                                                 
 conv_lstm2d_17 (ConvLSTM2D)  (None, 26, 26, 8, 20)    28880     
                                                                 
 max_pooling3d_17 (MaxPoolin  (None, 26, 26, 4, 20)    0         
 g3D)                                                            
                                                                 
 conv_lstm2d_18 (ConvLSTM2D)  (None, 26, 26, 4, 20)    2888

In [70]:
model.compile(
    loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),
)
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 1
batch_size = 5

# Fit the model to the training data.
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)

892/892 [==============================] - 948s 1s/step - loss: 0.1028 - val_loss: 0.1091 - lr: 0.0010


In [71]:
y_pred=model.predict(X_test)

In [72]:
print(y_test.shape)
print(y_pred.shape)
# y_pred=np.squeeze(y_pred,axis=-1)
# y_pred=np.squeeze(y_pred,axis=-1)
# y_test=np.squeeze(y_test,axis=-1)
# y_test=np.squeeze(y_test,axis=-1)

(1393, 26, 26, 6)
(1393, 26, 26, 1, 1)


In [73]:
y_pred=np.squeeze(y_pred,axis=-1)
print(y_pred.shape)

(1393, 26, 26, 1)


In [ ]:
threshold=np.percentile(y_pred,96)
def transform(x):
    return x>=threshold

In [ ]:
y_pred=transform(y_pred)
y_pred=y_pred.astype(int)

In [ ]:
y_test
y_pred

In [ ]:
from sklearn.metrics import precision_score, accuracy_score,recall_score,f1_score

accuracy_score(y_test.flatten(), y_pred.flatten()),precision_score(y_test.flatten(), y_pred.flatten()),recall_score(y_test.flatten(), y_pred.flatten()),f1_score(y_test.flatten(), y_pred.flatten())